# TP : Compression d'image avec k-means

Dans ce TP, vous allez découvrir une application pratique de l'algorithme K-means : **la compression d'images par réduction du nombre de couleurs**.

Vous apprendrez à :
* Implémenter une segmentation (clustering) k-means avec Scikit-learn
* Manipuler des images numériques avec NumPy et Scikit-image
* Appliquer k-means pour regrouper les couleurs similaires
* Analyser l'impact du nombre de clusters sur la qualité et la compression
* Déterminer le niveau de compression optimal

## Principe de la compression par k-means

Une image couleur est composée de pixels, chacun ayant une couleur représentée par 3 valeurs RGB (Rouge, Vert, Bleu). Une image peut contenir des milliers de couleurs différentes.

**L'idée** : utiliser K-means pour regrouper les couleurs similaires et remplacer chaque pixel par la couleur moyenne de son cluster. On réduit ainsi le nombre de couleurs de l'image, ce qui permet de la compresser.

## k-means avec Scikit-learn

Note : Si vous avez déjà exploré la manière de réaliser k-means avec Scikit-learn (questions de fin du cours), vous pouvez passer cette partie et démarrer directement le TP sur les images, ou juste – par acquis de conscience – la lire rapidement.

### Générer des données

Dans le cours nous avons évoqué la méthode `.make_blobs()` pour générer des clusters de données.

In [ ]:
#!pip install scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import make_blobs

points, labels = make_blobs(n_samples=500, centers=5, random_state=42)

print(f"\nLes 10 premiers points générés :\n{points[:10]}")
print(f"\nLes 10 premiers labels correspondants :\n{labels[:10]}")

#visualisation
# la couleur correspond aux labels, 
# coordonnées x première colonne 
# coordonnées y seconde colonne
plt.figure(figsize=(5, 5))
plt.scatter(x=points[:, 0], y=points[:, 1], c=labels);

Nous voyons que `.make_blob()` nous donne déjà les labels des clusters générés, hors avec k-means l’objectif est précisément de détecter automatiquement les clusters, nous allons donc ignorer les labels pour la suite.

Nous allons donc naïvement utiliser Scikit-learn pour extraire 2 clusters dans un premier temps.

### La classe `KMeans()`

* importée de `sklearn.cluster`
* paramètre `n_cluster` pour le nombre de clusters voulus
* paramètre `random_state` pour la reproductibilité (seed du générateur aléatoire)
* méthode `.fit_predict()` qui retourne la labellisation des clusters

In [ ]:
from sklearn.cluster import KMeans

labels_pred = KMeans(n_clusters=2, random_state=42).fit_predict(points)

labels_pred est un vecteur donc chaque index est le label du cluster prédit pour l’observation correspondante (comem la variable ̀ labels` ci-dessus). Les labels sont des entiers, ici 0 ou 1 car on a demandé 2 clusters :

In [ ]:
labels_pred[:10]

In [ ]:
# visualisation  
plt.figure(figsize=(5, 5))
plt.scatter(points[:, 0], points[:, 1], c=labels_pred)
plt.title("Identification de deux clusters");

### Pister l’inertie et choisir le bon nombre de clusters

Lors que l’on ajuste le nombre de cluster, une propriété `.inertia_` est créée pour l’objet `KMeans`.

In [ ]:
# la propriété n’existe pas avant le .fit()
test_inertia = KMeans(n_clusters=2, random_state=42)
test_inertia.inertia_

In [ ]:
# Elle existe bien après le .fit()
test_inertia = KMeans(n_clusters=2, random_state=42).fit(points)
test_inertia.inertia_

Rappel : l’inertie est la somme des carrés des écarts entre la position d’une observation et du centroïde le plus proche. C’est la grandeur que nous allons essayer de minimiser pour optimiser le modèle k-means. Cela peut être vu comme l’erreur de prédiction du modèle (pour une observation donnée, le modèle va prédire une valeur correspondante à celle du centroïde le plus proche), et on veut minimiser cette erreur. C’est le principe général de toute méthode d’apprentissage.

Pour trouver le nombre de clusters optimal, on va chercher le nombre à partir duquel ajouter des clusters ne diminue l’inertie (l’erreur) que marginalement : graphiquement, c’est la méthode du coude (que nous avons vue en cours).

### Exercice : méthode du coude

* créez une liste vide `inerties`
* crééz une liste `clusters` du nombre de clusters envisagés (p. ex. de 1 à 10)
* calculer l’inertie pour chaque nombre de clusters et stockez la dans la liste
* affichez sur un graphe l’inertie en fonction du nombre de clusters
* déterminez le nombre de clusters qui vous semble optimal

In [ ]:
# Exercice : votre code !


In [ ]:
# visualisation



On voit que k-mean a du mal à discriminer les deux clusters très proches à droite, mais on voit que l’inertie ne diminue encore un tout petit peu après 4 clusters, et plus du tout après 5 clusters.

Pour bien voir la différence, on peut calculer le ratio entre l’inertie pour un nombre de cluster n, et celle pour un nombre de cluster n-1 : on voit ainsi en proportion comment l’inertie diminue à chaque fois qu’on ajoute un cluster, ce qui rend la décision plus facile à identifier. C’est ce que l’on peut appeler « taux de décroissance de l’inertie ».

In [ ]:
# afficher le taux de décroissance de l’inertie

plt.plot(range(1, 10), np.array(inerties[1:])/np.array(inerties[:-1]))
plt.title('Taux de décroissance de l’inertie à chaque fois qu’on ajoute un cluster');

On voit clairement ici que 5 est bien le meilleur choix !

Maintenant que vous êtes familiarisé-e avec l’utilisation de Scikit-learn pour réaliser un k-mean, passons à une application concrète : la compression d’image

## 1. Imports et chargement de l'image

Installez le module `scikit-image`. Ici nous ne l’utiliserons que pour disposer d’une image standard, mais nous utiliserons cette bibliothèque plus tard dans le cours (preprocessing d’image, filtres automorphiques, etc.), autant l’installer d’ores et déjà dans votre environnement virtuel.

In [ ]:
#!pip install scikit-image

In [ ]:
# Imports nécessaires
import numpy as np
import matplotlib.pyplot as plt
from skimage import data
from sklearn.cluster import KMeans

# Fixons la taille des figures affichées
plt.rcParams['figure.figsize'] = (12, 8)

### 1.1 Charger l'image

Nous utilisons l'image `cat()` du module `data` de scikit-image.

In [ ]:
# Chargement de l'image
image = data.cat()

print(f"Type de l'objet : {type(image)}")
print(f"Type des données : {image.dtype}")

### 1.2 Afficher l'image avec Matplotlib

La méthode à utiliser est `.imshow()`. À vous de jouer :

In [ ]:
# Affichage de l'image originale
plt.figure(figsize=(10, 6))

# VOTRE CODE ICI

plt.title("Image originale")
plt.axis('off')
plt.show()

### 1.3 Analyser l'image avec NumPy

Une image est représentée comme un tableau NumPy à 3 dimensions : (hauteur, largeur, canaux)

Indiquez donc les caractéristiques de l’image chargée :
* ses dimensions
* sa hauteur en pixels
* sa largeur en pixels
* le nombrede canaux/couleurs
* le nombre total de pixels qu’elle contient
* les valeurs (« couleur ») minimales et maximales observées pour un pixel dans l’image
* la plage de valeur possible (d’après le type de la variable qui code la valeur d’une pixel)
* quel est le nombre de couleurs différentes (uniques) dans l’image
* quel est le nombre potentiel de couleur différente (à déduire de la plage de valeurs possibles vu ci-dessus) 

In [ ]:
# Dimensions de l'image


# Plage de valeurs


In [ ]:
# Nombre de couleurs uniques dans l'image
# On transforme l'image en liste de couleurs RGB
pixels = image.reshape(-1, 3)
couleurs_uniques =  # VOTRE CODE ICI
nb_couleurs =  # VOTRE CODE ICI

## 2. Préparation des données pour K-means

K-means attend des données sous forme de matrice 2D où chaque ligne est une observation.

**Question** : Quelle transformation devons-nous appliquer à notre image ? Indice : la méthode `.reshape()` nous sera très utile ici.

Interprétez ce changement de dimension :
* À quoi correspondent les dimensions de la nd.array de l’image originale ?
* À quoi correspondent les dimensions de la nd.array après `.reshape()` ?

In [ ]:
# Transformation de l'image 3D en matrice 2D
# Chaque ligne = un pixel avec ses 3 valeurs RGB

pixels = # VOTRE CODE ICI

## 3. Application de K-means avec 32 couleurs

* Définissez le nombre de couleurs voulues
* Appliquez k-means avec le paramètre qui va bien

In [ ]:
# Nombre de clusters (couleurs) souhaités
n_colors = # VOTRE CODE ICI

# Application de K-means

kmeans = # VOTRE CODE ICI

## 4. Vérification des résultats de K-means

In [ ]:
# Nombre de clusters
print(f"Nombre de clusters créés : {kmeans.n_clusters}")

# Les labels : à quel cluster appartient chaque pixel
labels = kmeans.labels_
print(f"\nShape des labels : {labels.shape}")
print(f"Labels uniques : {np.unique(labels)}")

Indiquez :
* la dimension des centroïdes
* quel est leur datatype ?
* Mathématiquement, cela correspond à quoi (intervalle, valeur possible…) ?
* Affichez les premièrs centroïdes, en indiquant à chaque fois à quelle valeur de rouge, de vert et de bleu ils correspondent.

In [ ]:
# Les centroïdes : les couleurs représentatives de chaque cluster
centroids = kmeans.cluster_centers_

# VOTRE CODE ICI

### Visualisation de la palette de couleurs compressée

Affichez les 32 couleurs qui correspondent aux . Indice : utiliser la méthode `.reshape()` pour reconstruire une image affichable à partir des données `centroids`. Il faudra peut-être faire une conversion de type également…

In [ ]:
# Affichage des 32 couleurs de la palette
palette = # VOTRE CODE ICI

plt.figure(figsize=(12, 2))
plt.imshow(palette)
plt.title(f"Palette de {n_colors} couleurs extraites par K-means")
plt.axis('off')
plt.show()

## 5. Reconstruction de l'image compressée

Pour chaque pixel, on remplace sa couleur originale par le centroïde de son cluster.
* indiquez les dimensions de l’image compressée « telle quelle »
* faire un reshape pour avoir une image affichable
* indiquez les nouvelles dimensions de l’image compressée

In [ ]:
# Remplacement de chaque pixel par son centroïde
image_compressée = centroids[labels]

# VOTRE CODE ICI

Ici aussi il faut convertir le type des données de l’image compressée pour pouvoir afficher :

In [ ]:
# Conversion en uint8 pour l'affichage
image_compressée = image_compressée.astype('uint8')

print(f"Type des données : {image_compressée.dtype}")
print(f"Valeurs min/max : {image_compressée.min()} / {image_compressée.max()}")

## 6. Caractéristiques de l'image compressée

Comme précédemment, indiquez les caractéristiques de l’image compressée :
* mesurez le nombre de couleurs uniques de l’image compressée
* rappelez le nombre de couleurs de l’image originale
* indiquez le taux de compression ×N (= nombre de couleurs originales / nombre de couleurs compressées)
* indiquez la réduction en pourcentage R% (= 1 - nombre de couleurs compressées / nombre de couleurs originales)

In [ ]:
# Nombre de couleurs dans l'image compressée
pixels_compressés = # VOTRE CODE ICI
couleurs_uniques_compressées = # VOTRE CODE ICI

print("=" * 60)
print("COMPARAISON IMAGE ORIGINALE vs IMAGE COMPRESSÉE")
print("=" * 60)
print(f"\nImage originale :")
# VOTRE CODE ICI

print(f"\nImage compressée :")
# VOTRE CODE ICI

## 7. Affichage de l'image compressée

Affichez les deux images pour que l’on puisse comparer la qualité de la réduction à l’œil.

In [ ]:
# Comparaison visuelle
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(image)
axes[0].set_title(f"Image originale\n({nb_couleurs} couleurs)")
axes[0].axis('off')

axes[1].imshow(image_compressée)
axes[1].set_title(f"Image compressée\n({n_colors} couleurs)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 8. Fonction de compression générique

Créons une fonction réutilisable pour compresser n'importe quelle image avec un nombre de couleurs donné.

In [ ]:
def compression_kmeans(image, n_colors, afficher=True):
    """
    Compresse une image en réduisant le nombre de couleurs avec K-means.
    
    Paramètres:
    -----------
    image : numpy.ndarray
        Image à compresser (shape: hauteur × largeur × 3)
    n_colors : int
        Nombre de couleurs souhaitées dans l'image compressée
    afficher : bool
        Si True, affiche l'image compressée
    
    Retourne:
    ---------
    image_compressée : numpy.ndarray
        Image compressée
    kmeans : KMeans
        Le modèle KMeans entraîné (pour accéder à l'inertie, etc.)
    """
    # 1. Préparation des données
    pixels = # VOTRE CODE ICI
    
    # 2. Application de K-means
    kmeans = # VOTRE CODE ICI
    kmeans.fit(pixels)
    
    # 3. Reconstruction de l'image
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    image_compressée = # VOTRE CODE ICI
    
    # 4. Affichage si demandé
    if afficher:
        plt.figure(figsize=(8, 6))
        plt.imshow(image_compressée)
        plt.title(f"Image compressée - {n_colors} couleurs")
        plt.axis('off')
        plt.show()
    
    return image_compressée, kmeans

In [ ]:
# Test de la fonction
img_test, model_test = compression_kmeans(image, n_colors=16)

## 9. Test de différents niveaux de compression

Explorons visuellement l'impact du nombre de couleurs sur la qualité de l'image. Testez différents niveaux de compression (= nombre de couleurs). À votre avis combien de couleur est un bon compromis ?

In [ ]:
# Différents niveaux de compression à tester
niveaux_compression = [2, 4, 8, 16, 32, 64]

# Création d'une grille de sous-graphiques
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, n_colors in enumerate(niveaux_compression):
    print(f"Compression avec {n_colors} couleurs...")
    img_comp, _ = # VOTRE CODE ICI
    
    axes[idx].imshow(img_comp)
    axes[idx].set_title(f"{n_colors} couleurs")
    axes[idx].axis('off')

plt.suptitle("Impact du nombre de couleurs sur la qualité de l'image", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("\nTerminé !")

## 10. Courbe d'inertie : trouver le nombre de couleurs optimal

Nous avons vue en cours que l'**inertie** (ou *within-cluster sum of squares*) mesure la compacité des clusters. Plus elle est faible, mieux les pixels sont représentés par leurs centroïdes.

On cherche le "coude" de la courbe : le point où ajouter des couleurs n'améliore plus significativement la qualité.

Vous pouvez retrouver l’inertie d’un clustering avec la propriété `.inertia_` d’un objet KMeans.
* Pour un clustering avec un nombre de couleur donné, ajoutez à chaque fois la valeur de l’inertie correspondante à une liste `inerties`.
* Tracez ensuite un graphe `inerties` en fonction du nombre de couleurs
* Déterminez le nombre de couleurs qui est un bon compromis et comparez avec ce que vous avez choisi visuellement

In [ ]:
# Plage de valeurs à tester
range_n_colors = range(2, 65, 2)  # De 2 à 64 couleurs, par pas de 2

# Stockage des inerties
inerties = []

print("Calcul des inerties pour différents nombres de couleurs...")
for n in range_n_colors:
# VOTRE CODE ICI

print("Terminé !")

In [ ]:
# Tracé de la courbe d'inertie
plt.figure(figsize=(12, 6))
plt.plot(range_n_colors, inerties, 'b-o', linewidth=2, markersize=6)
plt.xlabel('Nombre de couleurs (clusters)', fontsize=12)
plt.ylabel('Inertie', fontsize=12)
plt.title('Courbe d\'inertie en fonction du nombre de couleurs\n(Méthode du coude)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Analyse de la courbe

1. Où se situe le "coude" de la courbe ?
2. Que représente l'inertie dans le contexte de la compression d'image ?
3. Quel compromis faisons-nous entre qualité et compression ?
4. Combien de couleurs recommanderiez-vous pour cette image ?

Pour vous aider, le bout de code ci-dessous affiche le taux de décroissance de l’inertie. Comment est-il calculé ? Comment l’interpréter ou l’utiliser pour décider du nombre de couleur à choisir ? Quelle limite fixez-vous ?

In [ ]:
# Calcul du taux de décroissance de l'inertie
# Pour identifier objectivement le coude
differences = np.diff(inerties)
taux_decroissance = np.abs(differences / inerties[:-1]) * 100

plt.figure(figsize=(12, 6))
plt.plot(list(range_n_colors)[1:], taux_decroissance, 'r-o', linewidth=2, markersize=6)
plt.xlabel('Nombre de couleurs', fontsize=12)
plt.ylabel('Taux de décroissance de l\'inertie (%)', fontsize=12)
plt.title('Taux de décroissance de l\'inertie\n(Utile pour identifier le coude)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

LIMITE = # VOTRE CHOIX
# Point où le taux de décroissance devient faible
idx_coude = np.where(taux_decroissance < LIMITE)[0][0] if any(taux_decroissance < LIMITE) else len(taux_decroissance)-1
n_optimal = list(range_n_colors)[1:][idx_coude]
print(f"\nNombre de couleurs suggéré (décroissance < {LIMITE}%) : {n_optimal}")

## 11. Comparaison finale avec le niveau optimal

In [ ]:
# Compression avec le niveau optimal suggéré
img_optimale, _ = compression_kmeans(image, n_optimal, afficher=False)

# Affichage comparatif
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(image)
axes[0].set_title(f"Image originale\n({nb_couleurs} couleurs)")
axes[0].axis('off')

axes[1].imshow(img_optimale)
axes[1].set_title(f"Image compressée (niveau optimal)\n({n_optimal} couleurs)")
axes[1].axis('off')

# Compression très forte pour comparaison
img_forte, _ = compression_kmeans(image, 8, afficher=False)
axes[2].imshow(img_forte)
axes[2].set_title("Image très compressée\n(8 couleurs)")
axes[2].axis('off')

plt.tight_layout()
plt.show()

## Conclusion

### Ce que nous avons appris

1. **K-means comme outil de compression** : K-means peut être utilisé pour réduire le nombre de couleurs d'une image en regroupant les couleurs similaires.

2. **Trade-off qualité/compression** : Plus on réduit le nombre de clusters, plus la compression est forte, mais plus on perd en qualité visuelle.

3. **Méthode du coude** : La courbe d'inertie permet d'identifier un nombre de couleurs optimal qui offre un bon compromis.

### Questions pour approfondir

1. **Complexité algorithmique** : Quel est l'impact du nombre de pixels sur le temps de calcul ?

2. **Initialisation** : Comment l'initialisation aléatoire de K-means peut-elle affecter le résultat ?

3. **Autres applications** : Quelles autres applications de K-means pouvez-vous imaginer ?

4. **Limitations** : Quelles sont les limitations de cette approche de compression ?

## Exercices supplémentaires (bonus)

### Exercice 1
Modifiez la fonction `compression_kmeans` pour qu'elle retourne également le temps d'exécution de l'algorithme.

### Exercice 2
Créez une fonction qui calcule l'erreur de reconstruction (MSE - Mean Squared Error) entre l'image originale et l'image compressée. Si vous n’êtes pas familier avec cet indice, ignorez cette question, nous y reviendrons en détail quand nous aborderons le machine learning et la régression.

### Exercice 3
Testez l'impact du paramètre `n_init` de K-means sur la qualité de la compression et le temps de calcul.

### Exercice 4
Chargez votre propre image et appliquez la compression. Comparez les résultats avec l'image du chat.

### Exercice 5
Plutôt que Scikit-learn, utilisez votre propre implémentation de l’algorithme k-means pour comparer.

### Pour aller plus loin

- Testez avec d'autres images
- Comparez avec d'autres algorithmes de clustering (Mean-Shift, DBSCAN)
- Explorez les méthodes de quantification des couleurs (palette adaptative)
- Mesurez la compression réelle en octets (pas seulement en nombre de couleurs)